# Data visualization of stock daily tables

In [2]:
import numpy as np
import glob
import pandas as pd
from bokeh.plotting import figure, show, gridplot
from bokeh.io import output_notebook
from math import pi
output_notebook()

Loading BokehJS ...

### Function definition for plotting 

In [3]:
def plot_candlestick(df: pd.DataFrame) -> None:
    inc = df["<CLOSE>"] > df["<OPEN>"]
    dec = df["<OPEN>"] > df["<CLOSE>"]
    w = 12*60*60*1000 # half day in ms

    TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,save"

    pp = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = df["<TICKER>"].values[0])
    ppv = figure(x_axis_type="datetime", x_range=pp.x_range, tools=TOOLS, plot_width=1000, plot_height=200, title="VOLUME")
    pp.xaxis.major_label_orientation = pi/4
    pp.xaxis.ticker.desired_num_ticks = 60
    pp.yaxis.ticker.desired_num_ticks = 20
    pp.grid.grid_line_alpha=0.3
    
    ppv.xaxis.major_label_orientation = pi/4
    ppv.xaxis.ticker.desired_num_ticks = 60
    ppv.yaxis.ticker.desired_num_ticks = 5
    ppv.grid.grid_line_alpha=0.3

    pp.segment(df.index, df["<HIGH>"], df.index, df["<LOW>"], color="black")
    pp.vbar(df.index[inc], w, df["<OPEN>"][inc], df["<CLOSE>"][inc], fill_color="#D5E1DD", line_color="black")
    pp.vbar(df.index[dec], w, df["<OPEN>"][dec], df["<CLOSE>"][dec], fill_color="#F2583E", line_color="black")
    ppv.vbar(df.index, w, 0, df["<VOL>"], fill_color="#F2583E", line_color="black")
    
    gg = gridplot([[pp], [ppv]])
    
    show(gg)  # open a browser

## Find proper file based on symbol parameter

In [4]:
all_files = glob.glob(r"..\data\*.mst")
symbol = "SANPL"
found_files = []
for path in all_files:
    if symbol in path:
        found_files.append(path)
found_files

['..\\data\\SANPL.mst', '..\\data\\SANPL2.mst']

## Load data using pandas and plot it

In [7]:
my_data = pd.read_csv(found_files[0], index_col=1)
my_data.index = pd.to_datetime(my_data.index, format="%Y%m%d")
my_data

,<TICKER>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
<DTYYYYMMDD>,,,,,,
2018-09-11,SANPL2,280.0,280.0,276.0,276.0,27
2018-09-12,SANPL2,276.0,280.0,276.0,280.0,69
2018-09-13,SANPL2,280.0,308.0,280.0,308.0,93
2018-09-14,SANPL2,310.0,330.0,310.0,330.0,136
2018-09-17,SANPL2,304.0,318.0,304.0,314.0,407
...,...,...,...,...,...,...
2019-06-24,SANPL2,366.0,378.0,362.0,378.0,84
2019-06-25,SANPL2,368.0,372.0,362.0,372.0,507
2019-06-26,SANPL2,364.0,368.0,364.0,366.0,187


In [6]:
plot_candlestick(my_data[-100:])